In [5]:
%pip install --pre -U "weaviate-client==4.*"
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import weaviate
import weaviate.classes as wvc

from weaviate.util import generate_uuid5
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

False

In [8]:
import weaviate

# url = os.getenv("WEAVIATE_URL")
# api_key = os.getenv("WEAVIATE_API_KEY")
url = "https://1syylnhesmc5shxvmydv1q.c1.us-east4.gcp.weaviate.cloud"
api_key = "C3kymue0kUBSuoh5tIwo24nzEr7mhxIOtexi"

client = weaviate.connect_to_wcs(
    cluster_url=url,
    auth_credentials=weaviate.AuthApiKey(api_key)
)

In [10]:
#create a Weaviate collection to hold PDF and PDFPages

pdfs_collection = client.collections.create(
    name="PDF_document",
    vectorizer_config=wvc.Configure.Vectorizer.none(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(
            name="title",
            description="Name of the document. If none, defaults to File Name w/o extension",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="file_name",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="leadership_score",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="page_count",
            data_type=wvc.DataType.INT
        ),
        wvc.Property(
            name="creation_date",
            data_type=wvc.DataType.DATE
        ),
        wvc.Property(
            name="effective_date",
            data_type=wvc.DataType.DATE
        ),
        wvc.Property(
            name="ingestion_date",
            data_type=wvc.DataType.DATE
        ),
        wvc.Property(
            name="expiration_date",
            data_type=wvc.DataType.DATE
        ),
        wvc.Property(
            name="curator",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="source",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="aux_specific",
            data_type=wvc.DataType.BOOL
        ),
        wvc.Property(
            name="organization",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="public_release",
            data_type=wvc.DataType.BOOL
        ),
        wvc.Property(
            name="publication_number",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="uuid",
            data_type=wvc.DataType.UUID
        )
    ]
)

pdf_pages_collection = client.collections.create(
    name="PDF_document_page",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(
            name="content",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="title",
            data_type=wvc.DataType.TEXT
        ),
        wvc.Property(
            name="uuid",
            data_type=wvc.DataType.UUID
        ),
        wvc.Property(
            name="page_number",
            data_type=wvc.DataType.INT
        ),
        wvc.Property(
            name="publication_number",
            data_type=wvc.DataType.TEXT,
            skip_vectorization=True
        ),
    ],
    references=[
        wvc.ReferenceProperty(
            name="hasPdfDocument",
            target_collection="PDF_document"
        )
    ]
)


In [ ]:
#This will retrieve a reference to the collection, which allows you to do operations on the collection
#https://weaviate.io/developers/weaviate/client-libraries/python#data
pdfs_collection = client.collections.get("PDF_document")



###
# Loop over all PDFs in the folder containing the PDFs
# for each individual PDF
    # fetch the metadata (creation_date, expiration_date etc..) 
    # create a Weaviate dataObject and insert into Weavaiate (first code snippet below)
    # call textSplitter on the PDF, retrieving a list of pages
    # for each page
        # create a Weaviate dataObject that holds all the information required for the page (see schema configured above)
        # add a reference to the object indexed above (by uuid, see reference below)
        # Index all pages into Weaviate (see code snippet below)
# Done <3 


#This will insert data for a single PDF (not the pages)
#this id is used in the next block to create a reference
pdf_id = generate_uuid5()
pdfs_collection.data.insert(
    weaviate.classes.DataObject(
        properties={
            "title": "",
            "file_name": "This is the file name",
            "creation_date": ""
            #...
        },
        uuid=pdf_id
    )
)



pdf_pages_collection = client.collections.get("PDF_document_page")

#this is pseudo code that would create a data object for the pages(chunks)
#We create a reference from the page, back to the original document
pages = []
for i in 5:
    pages.append(
        weaviate.classes.DataObject(
        properties={
            "title": "The title of the page",
            "page_number": i,
            "content": "The content of the page"
        },
        references={
            "hasPdfDocument": wvc.Reference.to(uuids=pdf_id) 
        },
        uuid=generate_uuid5()
    )
    )

pdf_pages_collection.data.insert_many(pages)

